<a href="https://colab.research.google.com/github/SelvamRaju/AI-ML-Capstone-Project-NLP-Chatbot/blob/DiptiParmar23-patch-1/GL_Capstone_ChatBot_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**NATURAL LANGUAGE PROCESSING - CAPSTONE PROJECT**

#**AIML-CHATBOT INTERFACE**

#**DOMAIN: Industrial safety. NLP based Chatbot**

• CONTEXT:
The database comes from one of the biggest industry in Brazil and in the world. It is an urgent need for industries/companies around the
globe to understand why employees still suffer some injuries/accidents in plants. Sometimes they also die in such environment

 DATA DESCRIPTION:
This The database is basically records of accidents from 12 different plants in 03 different countries which every line in the data is an
occurrence of an accident.
Columns description:
‣ Data: timestamp or time/date information
‣ Countries: which country the accident occurred (anonymised)
‣ Local: the city where the manufacturing plant is located (anonymised)
‣ Industry sector: which sector the plant belongs to
‣ Accident level: from I to VI, it registers how severe was the accident (I means not severe but VI means very severe)
‣ Potential Accident Level: Depending on the Accident Level, the database also registers how severe the accident could have been (due to other factors
involved in the accident)
‣ Gender: if the person is male of female
‣ Employee or Third Party: if the injured person is an employee or a third party
‣ Critical Risk: some description of the risk involved in the accident
‣ Description: Detailed description of how the accident happened.

• PROJECT OBJECTIVE:
Design a ML/DL based chatbot utility which can help the professionals to highlight the safety risk as per the incident description

#**‣ Input: Interim report**

#**Process: Step 1: Import the data**

In [3]:
#for reading data from google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

     / 25.9 MB 139 kB/s
     |████████████████████████████████| 10.9 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 73.1 MB/s 
     |████████████████████████████████| 102 kB 12.3 MB/s 
     |████████████████████████████████| 690 kB 70.1 MB/s 
     |████████████████████████████████| 4.7 MB 74.3 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 812 kB 73.2 MB/s 
     |████████████████████████████████| 38.1 MB 1.2 MB/s 
  Created wheel for pandas-profiling: filename=pandas_profiling-3.1.1-py2.py3-none-any.whl size=261287 sha256=38a67908b1e2b8bc3f79f0d810292adf77b58da92f4346ff468a407db2f9f764
  Stored in directory: /tmp/pip-ephem-wheel-cache-aln_lvsk/wheels/cc/d5/09/083fb07c9363a2f45854b0e3a7de7d7c560f07da74b9e9769d
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=cc2e47a6fc33abd3dcbbca6dea6be572f2c53d3943e5d08cf842b55864fe451c
  Stored in directory: /root/.cache/pip/wheels/70/e1/52/5b14

In [5]:
!pip install plotly --upgrade

     |████████████████████████████████| 27.7 MB 1.2 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0


In [6]:
!pip install plotly-express

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import calendar

import warnings 
warnings.filterwarnings('ignore')

import plotly
print(plotly.__version__)

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score,precision_score,f1_score,recall_score
from sklearn.preprocessing import MinMaxScaler

from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

import tensorflow as tf
from tensorflow.keras import regularizers,optimizers
from tensorflow.keras.layers import Activation,Dense

from pandas_profiling import ProfileReport

5.6.0


In [8]:
df_health=pd.read_csv("/content/drive/My Drive/AIML/Capstone Project Data/IHMStefanini_industrial_safety_and_health_database_with_accidents_description.csv")
df_health.head()

,Unnamed: 0,Data,Countries,Local,Industry Sector,Accident Level,Potential Accident Level,Genre,Employee or Third Party,Critical Risk,Description
0,0,2016-01-01 00:00:00,Country_01,Local_01,Mining,I,IV,Male,Third Party,Pressed,While removing the drill rod of the Jumbo 08 f...
1,1,2016-01-02 00:00:00,Country_02,Local_02,Mining,I,IV,Male,Employee,Pressurized Systems,During the activation of a sodium sulphide pum...
2,2,2016-01-06 00:00:00,Country_01,Local_03,Mining,I,III,Male,Third Party (Remote),Manual Tools,In the sub-station MILPO located at level +170...
3,3,2016-01-08 00:00:00,Country_01,Local_04,Mining,I,I,Male,Third Party,Others,Being 9:45 am. approximately in the Nv. 1880 C...
4,4,2016-01-10 00:00:00,Country_01,Local_04,Mining,IV,IV,Male,Third Party,Others,Approximately at 11:45 a.m. in circumstances t...


In [9]:
print("No. of Rows and Columns in Dataframe")
df_health.shape

No. of Rows and Columns in Dataframe


(425, 11)

In [10]:
print("No. of elements in Dataframe")
df_health.size

No. of elements in Dataframe


4675

In [11]:
prof = ProfileReport(df_health) 
prof

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

#**Step 2: Data cleansing**

In [12]:
df_health.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 425 entries, 0 to 424
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Unnamed: 0                425 non-null    int64 
 1   Data                      425 non-null    object
 2   Countries                 425 non-null    object
 3   Local                     425 non-null    object
 4   Industry Sector           425 non-null    object
 5   Accident Level            425 non-null    object
 6   Potential Accident Level  425 non-null    object
 7   Genre                     425 non-null    object
 8   Employee or Third Party   425 non-null    object
 9   Critical Risk             425 non-null    object
 10  Description               425 non-null    object
dtypes: int64(1), object(10)
memory usage: 36.6+ KB


In [13]:
df_health.isnull().values.any()

False

In [14]:
df_health.isna().values.any()

False

In [15]:
df_health.isnull().sum()

Unnamed: 0                  0
Data                        0
Countries                   0
Local                       0
Industry Sector             0
Accident Level              0
Potential Accident Level    0
Genre                       0
Employee or Third Party     0
Critical Risk               0
Description                 0
dtype: int64

In [16]:
df_health.isna().sum()

Unnamed: 0                  0
Data                        0
Countries                   0
Local                       0
Industry Sector             0
Accident Level              0
Potential Accident Level    0
Genre                       0
Employee or Third Party     0
Critical Risk               0
Description                 0
dtype: int64

In [17]:
df_health.duplicated().sum()

0

In [18]:
df_health.describe(include=['object'])

,Data,Countries,Local,Industry Sector,Accident Level,Potential Accident Level,Genre,Employee or Third Party,Critical Risk,Description
count,425,425,425,425,425,425,425,425,425,425
unique,287,3,12,3,5,6,2,3,33,411
top,2017-02-08 00:00:00,Country_01,Local_03,Mining,I,IV,Male,Third Party,Others,During the activity of chuteo of ore in hopper...
freq,6,251,90,241,316,143,403,189,232,3


In [19]:
#Dropping this column as it will be 1-1 mapping and not usefule while training or testing the model
df_health=df_health.drop('Unnamed: 0',axis=1)
df_health.head()

,Data,Countries,Local,Industry Sector,Accident Level,Potential Accident Level,Genre,Employee or Third Party,Critical Risk,Description
0,2016-01-01 00:00:00,Country_01,Local_01,Mining,I,IV,Male,Third Party,Pressed,While removing the drill rod of the Jumbo 08 f...
1,2016-01-02 00:00:00,Country_02,Local_02,Mining,I,IV,Male,Employee,Pressurized Systems,During the activation of a sodium sulphide pum...
2,2016-01-06 00:00:00,Country_01,Local_03,Mining,I,III,Male,Third Party (Remote),Manual Tools,In the sub-station MILPO located at level +170...
3,2016-01-08 00:00:00,Country_01,Local_04,Mining,I,I,Male,Third Party,Others,Being 9:45 am. approximately in the Nv. 1880 C...
4,2016-01-10 00:00:00,Country_01,Local_04,Mining,IV,IV,Male,Third Party,Others,Approximately at 11:45 a.m. in circumstances t...


In [20]:
# Rename 'Data', 'Countries', 'Genre', 'Employee or Third Party' columns in Data frame
df_health.rename(columns={'Data':'Date','Countries':'Country','Genre':'Gender','Employee or Third Party':'Employee Type'},inplace=True)
df_health.head()

,Date,Country,Local,Industry Sector,Accident Level,Potential Accident Level,Gender,Employee Type,Critical Risk,Description
0,2016-01-01 00:00:00,Country_01,Local_01,Mining,I,IV,Male,Third Party,Pressed,While removing the drill rod of the Jumbo 08 f...
1,2016-01-02 00:00:00,Country_02,Local_02,Mining,I,IV,Male,Employee,Pressurized Systems,During the activation of a sodium sulphide pum...
2,2016-01-06 00:00:00,Country_01,Local_03,Mining,I,III,Male,Third Party (Remote),Manual Tools,In the sub-station MILPO located at level +170...
3,2016-01-08 00:00:00,Country_01,Local_04,Mining,I,I,Male,Third Party,Others,Being 9:45 am. approximately in the Nv. 1880 C...
4,2016-01-10 00:00:00,Country_01,Local_04,Mining,IV,IV,Male,Third Party,Others,Approximately at 11:45 a.m. in circumstances t...


 # **Step 3: Data preprocessing**

In [21]:
print(df_health.Date.max())
print(df_health.Date.min())

2017-07-09 00:00:00
2016-01-01 00:00:00


The whole data set is between 1st January 2016 to 9th July 2017


In [22]:
df_health['Date']=pd.to_datetime(df_health['Date'])
df_health.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 425 entries, 0 to 424
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Date                      425 non-null    datetime64[ns]
 1   Country                   425 non-null    object        
 2   Local                     425 non-null    object        
 3   Industry Sector           425 non-null    object        
 4   Accident Level            425 non-null    object        
 5   Potential Accident Level  425 non-null    object        
 6   Gender                    425 non-null    object        
 7   Employee Type             425 non-null    object        
 8   Critical Risk             425 non-null    object        
 9   Description               425 non-null    object        
dtypes: datetime64[ns](1), object(9)
memory usage: 33.3+ KB


In [23]:
replace_values = {'Local_01': 1, 'Local_02': 2, 'Local_03': 3, 'Local_04': 4, 'Local_05': 5, 'Local_06': 6, 'Local_07': 7, 'Local_08': 8, 'Local_09': 9, 'Local_10': 10, 'Local_11': 11, 'Local_12': 12}
df_health['Local'] = df_health['Local'].map(replace_values)
replace_values = {'I': 1, 'II': 2, 'III': 3, 'IV': 4, 'V': 5}
df_health['Accident Level'] = df_health['Accident Level'].map(replace_values)
replace_values = {'I': 1, 'II': 2, 'III': 3, 'IV': 4, 'V': 5, 'VI': 6}
df_health['Potential Accident Level'] = df_health['Potential Accident Level'].map(replace_values)
del replace_values

In [24]:
print(f'Dataset after adjusting categorical values\n')
df_health.head()

Dataset after adjusting categorical values



,Date,Country,Local,Industry Sector,Accident Level,Potential Accident Level,Gender,Employee Type,Critical Risk,Description
0,2016-01-01,Country_01,1,Mining,1,4,Male,Third Party,Pressed,While removing the drill rod of the Jumbo 08 f...
1,2016-01-02,Country_02,2,Mining,1,4,Male,Employee,Pressurized Systems,During the activation of a sodium sulphide pum...
2,2016-01-06,Country_01,3,Mining,1,3,Male,Third Party (Remote),Manual Tools,In the sub-station MILPO located at level +170...
3,2016-01-08,Country_01,4,Mining,1,1,Male,Third Party,Others,Being 9:45 am. approximately in the Nv. 1880 C...
4,2016-01-10,Country_01,4,Mining,4,4,Male,Third Party,Others,Approximately at 11:45 a.m. in circumstances t...


In [25]:
df_health.isnull().sum()

Date                        0
Country                     0
Local                       0
Industry Sector             0
Accident Level              0
Potential Accident Level    0
Gender                      0
Employee Type               0
Critical Risk               0
Description                 0
dtype: int64

Accidents happen throughout the year added datetime features such as year,month and day.

In [26]:
df_health['Date'] = pd.to_datetime(df_health['Date'])
df_health['Year'] = df_health['Date'].apply(lambda x : x.year)
df_health['Month'] = df_health['Date'].apply(lambda x : x.month)
df_health['Day'] = df_health['Date'].apply(lambda x : x.day)
df_health['Weekday'] = df_health['Date'].apply(lambda x : x.day_name())
df_health['WeekofYear'] = df_health['Date'].apply(lambda x : x.weekofyear)
df_health.head()

,Date,Country,Local,Industry Sector,Accident Level,Potential Accident Level,Gender,Employee Type,Critical Risk,Description,Year,Month,Day,Weekday,WeekofYear
0,2016-01-01,Country_01,1,Mining,1,4,Male,Third Party,Pressed,While removing the drill rod of the Jumbo 08 f...,2016,1,1,Friday,53
1,2016-01-02,Country_02,2,Mining,1,4,Male,Employee,Pressurized Systems,During the activation of a sodium sulphide pum...,2016,1,2,Saturday,53
2,2016-01-06,Country_01,3,Mining,1,3,Male,Third Party (Remote),Manual Tools,In the sub-station MILPO located at level +170...,2016,1,6,Wednesday,1
3,2016-01-08,Country_01,4,Mining,1,1,Male,Third Party,Others,Being 9:45 am. approximately in the Nv. 1880 C...,2016,1,8,Friday,1
4,2016-01-10,Country_01,4,Mining,4,4,Male,Third Party,Others,Approximately at 11:45 a.m. in circumstances t...,2016,1,10,Sunday,1


Dividing the year in quaters

In [27]:
def month_quarter_Conversion(x):
    if x in [10, 11, 12]:
        season = 'Fourth'
    elif x in [1, 2, 3]:
        season = 'First'
    elif x in [4, 5, 6]:
        season = 'Second'
    elif x in [7, 8, 9]:
        season = 'Third'
    return season
df_health['Quarter'] = df_health['Month'].apply(month_quarter_Conversion)
df_health.head()

,Date,Country,Local,Industry Sector,Accident Level,Potential Accident Level,Gender,Employee Type,Critical Risk,Description,Year,Month,Day,Weekday,WeekofYear,Quarter
0,2016-01-01,Country_01,1,Mining,1,4,Male,Third Party,Pressed,While removing the drill rod of the Jumbo 08 f...,2016,1,1,Friday,53,First
1,2016-01-02,Country_02,2,Mining,1,4,Male,Employee,Pressurized Systems,During the activation of a sodium sulphide pum...,2016,1,2,Saturday,53,First
2,2016-01-06,Country_01,3,Mining,1,3,Male,Third Party (Remote),Manual Tools,In the sub-station MILPO located at level +170...,2016,1,6,Wednesday,1,First
3,2016-01-08,Country_01,4,Mining,1,1,Male,Third Party,Others,Being 9:45 am. approximately in the Nv. 1880 C...,2016,1,8,Friday,1,First
4,2016-01-10,Country_01,4,Mining,4,4,Male,Third Party,Others,Approximately at 11:45 a.m. in circumstances t...,2016,1,10,Sunday,1,First


In [28]:
plt.figure(figsize=(5,4))
plt.subplot(1,2,1)
plt.title('Potential Accident Level')
sns.countplot(x='Potential Accident Level',data=df_health)

plt.subplot(1,2,2)
plt.title('Accident Level')
sns.countplot(x='Accident Level',data=df_health)

In [29]:
fig=plt.figure(figsize=(5,5))

ax1=fig.add_subplot(1,2,1)
plt.pie(x=df_health['Accident Level'].value_counts(),autopct='%.2f',labels=['1','2','3','4','5'])
plt.title('Accident Level',fontsize=14)

ax1=fig.add_subplot(1,2,2)
plt.pie(x=df_health['Potential Accident Level'].value_counts(),autopct='%.2f',labels=['1','2','3','4','5','6'])
plt.title('Potential Accident Level',fontsize=14)

Text(0.5, 1.0, 'Potential Accident Level')

In [30]:
#Dealing with categorical variable
#check for unique values in target varibale.
#It contains six values :
df_health['Potential Accident Level'].unique()

array([4, 3, 1, 2, 5, 6])

In [31]:
#check frequency distribution of target variable. 
#check whether data is balanced or imbalanced.
#It has 1 severe case and 49 not severe
df_health['Potential Accident Level'].value_counts()

4    143
3    106
2     95
1     49
5     31
6      1
Name: Potential Accident Level, dtype: int64

In [32]:
#We can check the imbalaned data by caluclating percentage of the classes
df_health['Potential Accident Level'].value_counts(normalize=True)

4    0.336471
3    0.249412
2    0.223529
1    0.115294
5    0.072941
6    0.002353
Name: Potential Accident Level, dtype: float64

In [33]:
df_health['Gender'].value_counts()

Male      403
Female     22
Name: Gender, dtype: int64

In [34]:
df_health['Industry Sector'].value_counts()

Mining    241
Metals    134
Others     50
Name: Industry Sector, dtype: int64

In [35]:
df_health['Employee Type'].value_counts()

Third Party             189
Employee                179
Third Party (Remote)     57
Name: Employee Type, dtype: int64

In [36]:

df_health['Accident Level'].value_counts()

1    316
2     40
3     31
4     30
5      8
Name: Accident Level, dtype: int64

In [37]:
df_health['Critical Risk'].value_counts()

Others                                       232
Pressed                                       24
Manual Tools                                  20
Chemical substances                           17
Venomous Animals                              16
Cut                                           14
Projection                                    13
Bees                                          10
Fall                                           9
Vehicles and Mobile Equipment                  8
Fall prevention (same level)                   7
remains of choco                               7
Pressurized Systems                            7
Fall prevention                                6
Suspended Loads                                6
Blocking and isolation of energies             3
Pressurized Systems / Chemical Substances      3
Power lock                                     3
Liquid Metal                                   3
Projection of fragments                        2
Machine Protection  

In [38]:
df_health['Country'].value_counts()

Country_01    251
Country_02    130
Country_03     44
Name: Country, dtype: int64

In [39]:
df_health['Local'].value_counts()

3     90
5     59
1     57
4     56
6     46
10    44
8     27
2     24
7     14
12     4
9      2
11     2
Name: Local, dtype: int64

In [40]:
plt.figure(figsize=(20,4))
plt.subplot(1,4,1)
plt.title("Country")
sns.countplot(x='Country',data=df_health)

plt.subplot(1,4,2)
plt.title("Local")
sns.countplot(x='Local',data=df_health)

plt.subplot(1,4,3)
plt.title("Industry Sector")
sns.countplot(x='Industry Sector',data=df_health)

plt.subplot(1,4,4)
plt.title("Year")
sns.countplot(x='Year',data=df_health)

plt.figure(figsize=(20,4))
plt.subplot(1,4,1)
plt.title("Month")
sns.countplot(x='Month',data=df_health)

plt.subplot(1,4,2)
plt.title("Gender")
sns.countplot(x='Gender',data=df_health)

plt.subplot(1,4,3)
plt.title("Employee Type")
sns.countplot(x='Employee Type',data=df_health)

plt.subplot(1,4,4)
plt.title("Critical Risk")
sns.countplot(x='Critical Risk',data=df_health)

In [41]:
fig=plt.figure(figsize=(25,15))

ax1=fig.add_subplot(2,4,1)
plt.pie(x=df_health['Country'].value_counts(),autopct="%.2f",labels=df_health['Country'].value_counts().keys())
plt.title("origin Distribution",fontsize=14)
plt.title('Country',size=14)

ax2=fig.add_subplot(2,4,2)
plt.pie(x=df_health['Local'].value_counts(),autopct="%.2f",labels=df_health['Local'].value_counts().keys())
plt.title("origin Distribution",fontsize=14)
plt.title('Local',size=14)

ax3=fig.add_subplot(2,4,3)
plt.pie(x=df_health['Industry Sector'].value_counts(),autopct="%.2f",labels=df_health['Industry Sector'].value_counts().keys())
plt.title("origin Distribution",fontsize=14)
plt.title('Industry Sector',size=14)

ax4=fig.add_subplot(2,4,4)
plt.pie(x=df_health['Gender'].value_counts(),autopct="%.2f",labels=df_health['Gender'].value_counts().keys())
plt.title("origin Distribution",fontsize=14)
plt.title('Gender',size=14)

ax5=fig.add_subplot(2,4,5)
plt.pie(x=df_health['Employee Type'].value_counts(),autopct="%.2f",labels=df_health['Employee Type'].value_counts().keys())
plt.title("origin Distribution",fontsize=14)
plt.title("Employee Type",fontsize=14)

ax6=fig.add_subplot(2,4,6)
plt.pie(x=df_health['Year'].value_counts(),autopct="%.2f",labels=df_health['Year'].value_counts().keys())
plt.title("origin Distribution",fontsize=14)
plt.title('Year',size=14)

ax7=fig.add_subplot(2,4,7)
plt.pie(x=df_health['Month'].value_counts(),autopct="%.2f",labels=df_health['Month'].value_counts().keys())
plt.title("origin Distribution",fontsize=14)
plt.title('Month',size=14)

plt.show()

# Multivariate Analysis

In [42]:
sns.catplot(data=df_health, x="Potential Accident Level",col="Accident Level",kind="count")

In [43]:
fig2 = plt.figure(figsize = (15, 7.2))
ax1 = fig2.add_subplot(122)
sns.countplot(x = "Accident Level", data = df_health, ax = ax1, orient = 'v',hue = 'Potential Accident Level').set_title("Accident Level".capitalize() +' count plot by Potential Accident Level',fontsize = 13)
plt.legend(labels = df_health['Potential Accident Level'].unique())
plt.xticks(rotation = 90)

(array([0, 1, 2, 3, 4]), <a list of 5 Text major ticklabel objects>)

In [44]:
sns.catplot(data=df_health, x="Accident Level",col="Quarter",kind="count")

In [45]:
fig=sns.FacetGrid(df_health,aspect=1.5,palette="winter", hue='Gender',col='Industry Sector', legend_out=True)
fig.map(sns.histplot, 'Accident Level')
fig.add_legend()

In [46]:
# Define a common function to plot multivariate countplot
def plot_multi(df, col1, hue):
    fig = plt.figure(figsize = (8, 4))
    ax = fig.add_subplot(121)
    sns.countplot(x = col1, data = df, ax = ax, orient = 'v',
                  hue = hue).set_title(col1.capitalize() +' count plot by '+ hue, 
                                                                      fontsize = 13)
    plt.legend(labels = df[hue].unique())
    #plt.xticks(rotation = 90)
    
    return plt.show()

In [47]:
plot_multi(df_health, 'Quarter', 'Accident Level')

1.   Q2 had the highest Accident Level of severity-I. 
2.   No accidents have happened in the Q4

In [48]:
plot_multi(df_health, 'Potential Accident Level', 'Accident Level')

Always the actual Accident Level has been less than the potential one. So looks like there have been mitigation steps have been taken

#Analysis of Gender with Accident/ Potential Accident Level



In [49]:
plot_multi(df_health,  "Accident Level",  'Gender')
# Looking at the graph below more male candidates are prone to accidents compared to females

In [50]:
plot_multi(df_health,  "Potential Accident Level",  'Gender')
#Compared to graph below more male have higher potential accident levels compared to females

# Employee type vs Accident and Potential Accident Level analysis

In [51]:
plot_multi(df_health,  "Potential Accident Level",  'Employee Type')
# Third Party(Onsite) working employees are more prone to potential accidents compared other two. 
# Third Party (Remote) working employees are less prone to potential accidents

In [52]:
plot_multi(df_health,  "Accident Level",  'Employee Type')
# Employees (Full time) are more prone to accidents compared other two in Accident Severity level I (0).
# For rest of the Accident Severity levels, Third Party employees have hit into accidents, more than rest of the two employee categories on an average.
# Third Party (Remote) working employees are less prone to accidents

# Industry Sector vs Accident and Potential Accident Level analysis

In [53]:
plot_multi(df_health,  "Accident Level",  'Industry Sector')
# Mining industry seems to have more accidents amongst all the other idustries
# Most critical severity incidents seems to have happened in Mining and Metal industries only

In [54]:
plot_multi(df_health,  "Potential Accident Level",  'Industry Sector')
# Mining industry seems to have more potential for Severity 4 accidents

# Country vs Accident Level analysis

In [55]:
plot_multi(df_health,  "Accident Level",  'Country')
# Country 1 has highest levels of accidents that has happened in the past
#	The most fatal accidents have happened in Country 1 only.
#	Country 1 and 2 share the same number of Sev2 accidents

In [56]:
plot_multi(df_health,  "Potential Accident Level",  'Country')

# Accident's monthwise analysis

In [57]:
plot_multi(df_health,  "Month",  'Accident Level')
#•	Most of the accidents have happened in the initial 6 months. From July onwards the number and the severity has been reduced
#•	Hence this is an important field to influence the potential accident level
#•	None of the fatal accidents have happened from August onwards.

In [58]:
plot_multi(df_health,  "Month",  'Potential Accident Level')
#	Similarly , the potentials of accidents is higher in the first 6 months of the year.
#	Month of Feb shows the highest number of Sev 2 category accident’s potential.
#	Month plays an important role in Potential Accident target level prediction

# Accident's weekday wise analysis

In [59]:
def plot_multi_big(df, col1, hue):
    fig = plt.figure(figsize = (10, 8))
    ax = fig.add_subplot()
    sns.countplot(x = col1, data = df, ax = ax, orient = 'v',
                  hue = hue).set_title(col1.capitalize() +' count plot by '+ hue, 
                                                                      fontsize = 13)
    plt.legend(labels = df[hue].unique())
    plt.xticks(rotation = 90)
    
    return plt.show()

In [60]:
plot_multi_big(df_health,  "Weekday",  'Potential Accident Level')
#	There is less potential of Sev 5 accidents on Monday and Sunday.
#	The most fatal accidents can happen only on Wednesday. (Looking back at the data – there is only one record in the data sheet on 25th Jan 2017 i.e. Wednesday on which the Potential of accident severity was 5 
plot_multi_big(df_health,  "WeekofYear",  'Potential Accident Level')

# Country vs Employee type analysis

In [61]:
sns.countplot(x="Country", data=df_health, hue='Employee Type')
#Country 1 has most third party contractors
#Country 3 has very less full time employees and no third party remote workers
# Country 2 has very less Thirdparty workers working on site. Most number of full time employees 

# Analyze Country and Industry Sector

In [62]:
sns.countplot(x="Country", data=df_health, hue='Industry Sector')
#	Country 1 has the highest Mining industries compared to others. Also Mining industry has 56 % of accidents reported from the report. Thus no wonder if Country 1 has the highest number of incidents reported
#	Country 3 has no Mining and Metal industries. Other industries have 11 % of accidents. Country 3 has 10 % of accidents from the reported ones.

In [63]:
sns.countplot(x="Country", data=df_health,hue="Gender")
# In all the countries male workers are the most. Hence no wonder if the accidents are happening more for Males
#	Country 3 does not have any female worker belonging to the given industries

# Analyze Employee Type and Gender

> Indented block



In [64]:
sns.countplot(x="Employee Type", data=df_health,hue="Gender")
#	In all the three employment categories there are more males and those are the most affected ones during these accidents

# Analyze the year , industry and more..

In [65]:
fig,axs = plt.subplots(nrows=2,ncols=2,figsize=(15,10))
sns.countplot(x=df_health['Year'],hue='Industry Sector',data=df_health,ax=axs[0][0])
sns.countplot(x=df_health['Year'],hue='Accident Level',data=df_health,ax=axs[0][1])
sns.countplot(x=df_health['Industry Sector'],hue='Accident Level',data=df_health,ax=axs[1][0])
sns.countplot(x=df_health['Accident Level'],hue='Industry Sector',data=df_health,ax=axs[1][1])

In [66]:
sns.pairplot(df_health,diag_kind='kde')

Observations:
    1. From above pair plots,we can see that many columns are correlated among them, this leads to multi-collinearity and many columns have long tail so that is the indication of outliers. 

In [67]:
fig=plt.subplots(figsize=(20,10))
sns.heatmap(df_health.corr(),annot=True,linewidth=0.2)

In [68]:
corre_values_target=df_health.corr()['Potential Accident Level']
pd.DataFrame(corre_values_target.sort_values(ascending=False))

,Potential Accident Level
Potential Accident Level,1.000000
Accident Level,0.502704
Day,0.005721
Year,-0.004081
WeekofYear,-0.033401
Month,-0.047853
Local,-0.441603


# Analyze the description data

In [69]:
#Number of characters present in each description sentence

df_health['Description'].str.len().hist()

Average word length is as below

In [70]:
df_health['Description'].str.split().apply(lambda x : [len(i) for i in x]). \
   map(lambda x: np.mean(x)).hist()

#Stop word analysis

In [71]:
import matplotlib.pyplot as plt
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
def plot_top_stopwords_barchart(text):
    
    stop=set(stopwords.words('english'))
    
    new= text.str.split()
    new=new.values.tolist()
    corpus=[word for i in new for word in i]
    from collections import defaultdict
    dic=defaultdict(int)
    for word in corpus:
        if word in stop:
            dic[word]+=1
            
    top=sorted(dic.items(), key=lambda x:x[1],reverse=True)[:10] 
    x,y=zip(*top)
    plt.bar(x,y)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [72]:
plot_top_stopwords_barchart(df_health['Description'])  

In [73]:
import re

import nltk

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from wordcloud import WordCloud, STOPWORDS

In [74]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [75]:
df_health['Description'][0]

'While removing the drill rod of the Jumbo 08 for maintenance, the supervisor proceeds to loosen the support of the intermediate centralizer to facilitate the removal, seeing this the mechanic supports one end on the drill of the equipment to pull with both hands the bar and accelerate the removal from this, at this moment the bar slides from its point of support and tightens the fingers of the mechanic between the drilling bar and the beam of the jumbo.'

In [76]:
#Removing of stopwords and stemming

ps = PorterStemmer()
sentences = df_health['Description']
corpus = []
for i in range(len(sentences)):
    sentences_1 = nltk.sent_tokenize(sentences[i])

   #words =  [nltk.word_tokenize(sentence) for sentence in sentences_1]
    # Preprocessing the data
    review = re.sub(r'\[[0-9]*\]',' ',sentences[i])
    review = re.sub('[^a-zA-Z]', ' ', review)

    review = re.sub(r'\s+',' ',review)
    review = review.lower()
    review = re.sub(r'\d',' ',review)
    review = re.sub(r'\s+',' ',review)
    review = review.split()

    
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

In [77]:
print(corpus[0])
#Look at stemming words e.g remov , mainten , proce etc are not making sense in this use case. Since this is chat bot application , we need more semantical  words

remov drill rod jumbo mainten supervisor proce loosen support intermedi central facilit remov see mechan support one end drill equip pull hand bar acceler remov moment bar slide point support tighten finger mechan drill bar beam jumbo


In [78]:
#Removing of stopwords and lemmatization

ps = PorterStemmer()
wordnet=WordNetLemmatizer()
sentences = df_health['Description']
corpus = []
for i in range(len(sentences)):
    # Preprocessing the data
    review = re.sub(r'\[[0-9]*\]',' ',sentences[i])
    review = re.sub('[^a-zA-Z]', ' ', review)

    review = re.sub(r'\s+',' ',review)
    review = review.lower()
    review = re.sub(r'\d',' ',review)
    review = re.sub(r'\s+',' ',review)
    review = review.split()

    review = [wordnet.lemmatize(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

wordcloud = WordCloud(
        background_color='white',
        stopwords=set(STOPWORDS),
        max_words=100,
        max_font_size=30, 
        scale=3,
        random_state=1)
    
wordcloud=wordcloud.generate(str(corpus))

fig = plt.figure(1, figsize=(12, 12))
plt.axis('off')
 
plt.imshow(wordcloud)
plt.show()

In [79]:
print(corpus)
#Look at lemmatized words e.g removing , maintenance , proceeds  etc are  making more sense now compared to stemming example above.

['removing drill rod jumbo maintenance supervisor proceeds loosen support intermediate centralizer facilitate removal seeing mechanic support one end drill equipment pull hand bar accelerate removal moment bar slide point support tightens finger mechanic drilling bar beam jumbo', 'activation sodium sulphide pump piping uncoupled sulfide solution designed area reach maid immediately made use emergency shower directed ambulatory doctor later hospital note sulphide solution gram liter', 'sub station milpo located level collaborator excavation work pick hand tool hitting rock flat part beak bounce hitting steel tip safety shoe metatarsal area left foot collaborator causing injury', 'approximately nv cx ob personnel begin task unlocking soquet bolt bhb machine penultimate bolt identified hexagonal head worn proceeding mr crist bal auxiliary assistant climb platform exert pressure hand dado key prevent coming bolt moment two collaborator rotate lever anti clockwise direction leaving key bolt

In [80]:
len(corpus)

425

# N Gram analysis

In [81]:
from sklearn.feature_extraction.text import CountVectorizer
from collections import  Counter

def plot_top_ngrams_barchart(text, n=2):
    stop=set(stopwords.words('english'))

    new= text.str.split()
    new=new.values.tolist()
    corpus=[word for i in new for word in i]

    def _get_top_ngram(corpus, n=None):
        vec = CountVectorizer(ngram_range=(n, n)).fit(corpus)
        bag_of_words = vec.transform(corpus)
        sum_words = bag_of_words.sum(axis=0) 
        words_freq = [(word, sum_words[0, idx]) 
                      for word, idx in vec.vocabulary_.items()]
        words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
        return words_freq[:10]

    top_n_bigrams=_get_top_ngram(text,n)[:10]
    x,y=map(list,zip(*top_n_bigrams))
    sns.barplot(x=y,y=x)
    
plot_top_ngrams_barchart(df_health['Description'],2)

In [82]:
plot_top_ngrams_barchart(df_health['Description'],3)

# TFIDF Feature



In [83]:
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import tokenize,stem

feature_df = pd.DataFrame()
for i in [1,2,3]:
    vec_tfidf = TfidfVectorizer(max_features=3000, norm='l2', stop_words='english', lowercase=True, use_idf=True, ngram_range=(i,i))
    X = vec_tfidf.fit_transform(corpus).toarray()
    tfs = pd.DataFrame(X, columns=["TFIDF_" + n for n in vec_tfidf.get_feature_names()])
    feature_df = pd.concat([feature_df, tfs], axis=1)
feature_df = pd.concat([df_health, feature_df], axis=1)
feature_df.head(3)
X.shape

(425, 3000)

In [84]:
feature_df.head(3)


,Date,Country,Local,Industry Sector,Accident Level,Potential Accident Level,Gender,Employee Type,Critical Risk,Description,...,TFIDF_zinc sheet adhered,TFIDF_zinc sheet cathode,TFIDF_zinc stuck enter,TFIDF_zinco rotary table,TFIDF_zn ingot struck,TFIDF_zn sheet lifting,TFIDF_zone intersection rpa,TFIDF_zone lifting container,TFIDF_zone moment bricklayer,TFIDF_zone supervise work
0,2016-01-01,Country_01,1,Mining,1,4,Male,Third Party,Pressed,While removing the drill rod of the Jumbo 08 f...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2016-01-02,Country_02,2,Mining,1,4,Male,Employee,Pressurized Systems,During the activation of a sodium sulphide pum...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2016-01-06,Country_01,3,Mining,1,3,Male,Third Party (Remote),Manual Tools,In the sub-station MILPO located at level +170...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [86]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score


feature_df['Country'] = LabelEncoder().fit_transform(feature_df['Country']).astype(np.int8)
feature_df['Local'] = LabelEncoder().fit_transform(feature_df['Local']).astype(np.int8)
feature_df['Industry Sector'] = LabelEncoder().fit_transform(feature_df['Industry Sector']).astype(np.int8)
feature_df['Accident Level'] = LabelEncoder().fit_transform(feature_df['Accident Level']).astype(np.int8)
feature_df['Potential Accident Level'] = LabelEncoder().fit_transform(feature_df['Potential Accident Level']).astype(np.int8)
feature_df['Gender'] = LabelEncoder().fit_transform(feature_df['Gender']).astype(np.int8)
feature_df['Employee Type'] = LabelEncoder().fit_transform(feature_df['Employee Type']).astype(np.int8)
feature_df['Critical Risk'] = LabelEncoder().fit_transform(feature_df['Critical Risk']).astype(np.int8)
feature_df['Weekday'] = LabelEncoder().fit_transform(feature_df['Weekday']).astype(np.int8)
feature_df['Quarter'] = LabelEncoder().fit_transform(feature_df['Quarter']).astype(np.int8)
feature_df.head(3)

,Date,Country,Local,Industry Sector,Accident Level,Potential Accident Level,Gender,Employee Type,Critical Risk,Description,...,TFIDF_zinc sheet adhered,TFIDF_zinc sheet cathode,TFIDF_zinc stuck enter,TFIDF_zinco rotary table,TFIDF_zn ingot struck,TFIDF_zn sheet lifting,TFIDF_zone intersection rpa,TFIDF_zone lifting container,TFIDF_zone moment bricklayer,TFIDF_zone supervise work
0,2016-01-01,0,0,1,0,3,1,1,20,While removing the drill rod of the Jumbo 08 f...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2016-01-02,1,1,1,0,3,1,0,21,During the activation of a sodium sulphide pum...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2016-01-06,0,2,1,0,2,1,2,15,In the sub-station MILPO located at level +170...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [87]:
feature_df.dtypes

Date                            datetime64[ns]
Country                                   int8
Local                                     int8
Industry Sector                           int8
Accident Level                            int8
                                     ...      
TFIDF_zn sheet lifting                 float64
TFIDF_zone intersection rpa            float64
TFIDF_zone lifting container           float64
TFIDF_zone moment bricklayer           float64
TFIDF_zone supervise work              float64
Length: 8713, dtype: object

Data preprocessing for 'Description' column

In [88]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [89]:

from nltk.corpus import stopwords
stopwords=set(stopwords.words('english'))

In [90]:
print('Converting description to lower case')
feature_df['Cleaned_Description']=feature_df['Description'].apply(lambda x:x.lower())

print('Removing special symbols and numbers')
feature_df['Cleaned_Description']=feature_df['Cleaned_Description'].apply(lambda x: re.sub(r'[!@#$(),"%*?:;_~+-/`0-9]+',' ',x))

print('Removing Stop words')
feature_df['Cleaned_Description']=feature_df['Cleaned_Description'].apply(lambda x: ' '.join([words for words in x.split() if words not in stopwords]))

print('Remove all the leading and trailing spaces from a string')
feature_df['Cleaned_Description']=feature_df['Cleaned_Description'].apply(lambda x: x.strip())

print('Removing multiple spaces between words')
feature_df['Cleaned_Description']=feature_df['Cleaned_Description'].apply(lambda x: re.sub(' +', ' ',x))

#Actual data before cleaning
print('Actual Data :{}'.format(feature_df['Description'][0]))

#Data after cleaning
print('Cleaned Data :{}'.format(feature_df['Cleaned_Description'][0]))

Converting description to lower case
Removing special symbols and numbers
Removing Stop words
Remove all the leading and trailing spaces from a string
Removing multiple spaces between words
Actual Data :While removing the drill rod of the Jumbo 08 for maintenance, the supervisor proceeds to loosen the support of the intermediate centralizer to facilitate the removal, seeing this the mechanic supports one end on the drill of the equipment to pull with both hands the bar and accelerate the removal from this, at this moment the bar slides from its point of support and tightens the fingers of the mechanic between the drilling bar and the beam of the jumbo.
Cleaned Data :removing drill rod jumbo maintenance supervisor proceeds loosen support intermediate centralizer facilitate removal seeing mechanic supports one end drill equipment pull hands bar accelerate removal moment bar slides point support tightens fingers mechanic drilling bar beam jumbo


In [91]:
# shift column 'Name' to first position
first_column=feature_df.pop('Cleaned_Description')
feature_df.insert(0,'Cleaned_Description',first_column)
feature_df.head(3)

,Cleaned_Description,Date,Country,Local,Industry Sector,Accident Level,Potential Accident Level,Gender,Employee Type,Critical Risk,...,TFIDF_zinc sheet adhered,TFIDF_zinc sheet cathode,TFIDF_zinc stuck enter,TFIDF_zinco rotary table,TFIDF_zn ingot struck,TFIDF_zn sheet lifting,TFIDF_zone intersection rpa,TFIDF_zone lifting container,TFIDF_zone moment bricklayer,TFIDF_zone supervise work
0,removing drill rod jumbo maintenance superviso...,2016-01-01,0,0,1,0,3,1,1,20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,activation sodium sulphide pump piping uncoupl...,2016-01-02,1,1,1,0,3,1,0,21,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,sub station milpo located level collaborator e...,2016-01-06,0,2,1,0,2,1,2,15,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [92]:
feature_df.drop(['Date','Description'],axis=1,inplace=True)

In [93]:
list(feature_df.columns.values.tolist())

['Cleaned_Description',
 'Country',
 'Local',
 'Industry Sector',
 'Accident Level',
 'Potential Accident Level',
 'Gender',
 'Employee Type',
 'Critical Risk',
 'Year',
 'Month',
 'Day',
 'Weekday',
 'WeekofYear',
 'Quarter',
 'TFIDF_abb',
 'TFIDF_abdomen',
 'TFIDF_able',
 'TFIDF_abratech',
 'TFIDF_abrupt',
 'TFIDF_abruptly',
 'TFIDF_absorbent',
 'TFIDF_absorbing',
 'TFIDF_abutment',
 'TFIDF_acc',
 'TFIDF_accelerate',
 'TFIDF_access',
 'TFIDF_accessing',
 'TFIDF_accessory',
 'TFIDF_accident',
 'TFIDF_accidentally',
 'TFIDF_accidently',
 'TFIDF_accommodate',
 'TFIDF_accommodated',
 'TFIDF_accompanied',
 'TFIDF_accompanying',
 'TFIDF_according',
 'TFIDF_accretion',
 'TFIDF_accumulated',
 'TFIDF_accumulates',
 'TFIDF_accumulating',
 'TFIDF_accumulation',
 'TFIDF_achieving',
 'TFIDF_acid',
 'TFIDF_acl',
 'TFIDF_acquisition',
 'TFIDF_acted',
 'TFIDF_action',
 'TFIDF_activated',
 'TFIDF_activates',
 'TFIDF_activation',
 'TFIDF_activity',
 'TFIDF_actuating',
 'TFIDF_adapted',
 'TFIDF_adapter

In [94]:
corre_values_target=feature_df.corr()['Potential Accident Level']
TFIDF = pd.DataFrame(corre_values_target.sort_values(ascending=False))

In [95]:
print(TFIDF)

                          Potential Accident Level
Potential Accident Level                  1.000000
Accident Level                            0.502704
TFIDF_operator                            0.236865
TFIDF_block                               0.175690
TFIDF_height                              0.174325
...                                            ...
TFIDF_reaction                           -0.303132
TFIDF_allergic                           -0.311660
TFIDF_sting                              -0.319689
Country                                  -0.400094
Local                                    -0.441603

[8711 rows x 1 columns]


#Model training

##**• Perform train-test split**

In [96]:
X = feature_df.drop(['Accident Level','Potential Accident Level'], axis = 1) # Considering all Predictors
y = feature_df['Potential Accident Level']

In [97]:
cv=CountVectorizer(ngram_range=(1,2),max_features=5000,stop_words='english',analyzer='word')
X=cv.fit_transform(feature_df.Cleaned_Description).toarray()

print("X.shape :",X.shape)
print("y.shape :",y.shape)

X.shape : (425, 5000)
y.shape : (425,)


In [98]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 1)

Following is the shape of Features(X) and Labels(y) for Training and Testing

In [99]:
print(X_train.shape)
print(y_train.shape)

(340, 5000)
(340,)


In [100]:
print(X_test.shape)
print(y_test.shape)

(85, 5000)
(85,)


#**Model 1:Naive Bayes**

In [101]:
mnb = MultinomialNB()
NB_model = mnb.fit(X_train,y_train)
nb_y_pred=NB_model.predict(X_test)

In [102]:
NB_train_score=NB_model.score(X_train,y_train)
print(NB_train_score)
NB_test_score=NB_model.score(X_test,y_test)
print(NB_test_score)

0.9764705882352941
0.4235294117647059


In [103]:
NBacc=round(accuracy_score(y_test,nb_y_pred)*100,2)
print('Test Accuracy Score : % :',NBacc)
precNB=precision_score(y_test,nb_y_pred,average='micro')
print('Precision Score',precNB)
f1NB=f1_score(y_test,nb_y_pred,average='micro')
print('F1 Score :',f1NB)
recallNB=recall_score(y_test,nb_y_pred,average='micro')
print('Recall Score:',recallNB)

Test Accuracy Score : % : 42.35
Precision Score 0.4235294117647059
F1 Score : 0.4235294117647059
Recall Score: 0.4235294117647059


In [104]:
print(classification_report(y_test,nb_y_pred))

              precision    recall  f1-score   support

           0       0.62      0.50      0.56        10
           1       0.47      0.28      0.35        25
           2       0.24      0.43      0.31        14
           3       0.50      0.55      0.52        33
           4       0.00      0.00      0.00         3

    accuracy                           0.42        85
   macro avg       0.37      0.35      0.35        85
weighted avg       0.44      0.42      0.42        85



In [105]:
confusn_m_NB= confusion_matrix(y_test,nb_y_pred)
print(confusn_m_NB)

[[ 5  1  2  2  0]
 [ 1  7  9  8  0]
 [ 0  3  6  5  0]
 [ 2  4  8 18  1]
 [ 0  0  0  3  0]]


#**Model 2: Logistic Regression**

In [106]:
LoR1=LogisticRegression(solver='liblinear')
LoR1.fit(X_train,y_train)
y_pred_LoR1=LoR1.predict(X_test)

In [107]:
#Model score or accuracy of logistic regression model
LoR_train_score=LoR1.score(X_train,y_train)
print(LoR_train_score)

LoR_test_score=LoR1.score(X_test,y_test)
print(LoR_test_score)

0.9970588235294118
0.4235294117647059


In [108]:
LRacc=round(accuracy_score(y_test,y_pred_LoR1)*100,2)
print('Test Accuracy Score : % :',LRacc)
precLR=precision_score(y_test,y_pred_LoR1,average='micro')
print('Precision Score',precLR)
f1LR=f1_score(y_test,y_pred_LoR1,average='micro')
print('F1 Score :',f1LR)
recallLR=recall_score(y_test,y_pred_LoR1,average='micro')
print('Recall Score:',recallLR)

Test Accuracy Score : % : 42.35
Precision Score 0.4235294117647059
F1 Score : 0.4235294117647059
Recall Score: 0.4235294117647059


In [109]:
print(classification_report(y_test,y_pred_LoR1))

              precision    recall  f1-score   support

           0       0.67      0.40      0.50        10
           1       0.47      0.32      0.38        25
           2       0.16      0.29      0.21        14
           3       0.56      0.61      0.58        33
           4       0.00      0.00      0.00         3

    accuracy                           0.42        85
   macro avg       0.37      0.32      0.33        85
weighted avg       0.46      0.42      0.43        85



In [110]:
confusn_m_LR = confusion_matrix(y_test,y_pred_LoR1)
print(confusn_m_LR)

[[ 4  1  2  3  0]
 [ 1  8  9  7  0]
 [ 0  6  4  4  0]
 [ 1  2  9 20  1]
 [ 0  0  1  2  0]]


#**Model 3:Random Forest**

In [111]:
RF_model=RandomForestClassifier()
RF_model.fit(X_train,y_train)
RF_pred=RF_model.predict(X_test)

In [112]:
RF_train_score=RF_model.score(X_train,y_train)
print(RF_train_score)
RF_test_score=RF_model.score(X_test,y_test)
print(RF_test_score)

0.9970588235294118
0.36470588235294116


In [113]:
RFacc=round(accuracy_score(y_test,RF_pred)*100,2)
print('Test Accuracy Score : % :',RFacc)
RFpre=precision_score(y_test,RF_pred,average='micro')
print('Precision Score',RFpre)
RF_f1=f1_score(y_test,RF_pred,average='micro')
print('F1 Score :',RF_f1)
RFrecall=recall_score(y_test,RF_pred,average='micro')
print('Recall Score:',RFrecall)

Test Accuracy Score : % : 36.47
Precision Score 0.36470588235294116
F1 Score : 0.36470588235294116
Recall Score: 0.36470588235294116


In [114]:
print(classification_report(y_test,RF_pred))

              precision    recall  f1-score   support

           0       1.00      0.40      0.57        10
           1       0.44      0.16      0.24        25
           2       0.15      0.43      0.22        14
           3       0.53      0.52      0.52        33
           4       0.00      0.00      0.00         3

    accuracy                           0.36        85
   macro avg       0.43      0.30      0.31        85
weighted avg       0.48      0.36      0.38        85



In [115]:
confusn_m_RF = confusion_matrix(y_test,RF_pred)
print(confusn_m_RF)

[[ 4  1  3  2  0]
 [ 0  4 16  5  0]
 [ 0  1  6  7  0]
 [ 0  3 13 17  0]
 [ 0  0  2  1  0]]


Overfitting Problem

#**Model 4: Random Forest with MinMaxScaler**

In [116]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler().fit(X_train)
X_train_MinMax=scaler.transform(X_train)
X_test_MinMax=scaler.transform(X_test)

In [117]:
RF2_SS_model=RandomForestClassifier()
RF2_SS_model.fit(X_train_MinMax,y_train)
RF2_minmax_pred=RF2_SS_model.predict(X_test_MinMax)

In [118]:
modelRF_SSTrain_Score=RF2_SS_model.score(X_train_MinMax,y_train)
modelRF_SSTest_Score=RF2_SS_model.score(X_test_MinMax,y_test)
print(modelRF_SSTrain_Score)
print(modelRF_SSTest_Score)

0.9970588235294118
0.38823529411764707


In [119]:
RF2_acc=round(accuracy_score(y_test,RF2_minmax_pred)*100,2)
print('Test Accuracy Score : % :',RF2_acc)
RF2_prec=precision_score(y_test,RF2_minmax_pred,average='micro')
print('Precision Score',RF2_prec)
RF2_f1=f1_score(y_test,RF2_minmax_pred,average='micro')
print('F1 Score :',RF2_f1)
RF2recall=recall_score(y_test,RF2_minmax_pred,average='micro')
print('Recall Score:',RF2recall)

Test Accuracy Score : % : 38.82
Precision Score 0.38823529411764707
F1 Score : 0.38823529411764707
Recall Score: 0.38823529411764707


In [120]:
print(classification_report(y_test,RF2_minmax_pred))

              precision    recall  f1-score   support

           0       0.80      0.40      0.53        10
           1       0.25      0.08      0.12        25
           2       0.21      0.50      0.29        14
           3       0.53      0.61      0.56        33
           4       0.00      0.00      0.00         3

    accuracy                           0.39        85
   macro avg       0.36      0.32      0.30        85
weighted avg       0.41      0.39      0.37        85



In [121]:
confusn_m_RFwithScaling = confusion_matrix(y_test,RF2_minmax_pred)
print(confusn_m_RFwithScaling)

[[ 4  1  2  3  0]
 [ 1  2 14  8  0]
 [ 0  3  7  4  0]
 [ 0  2 11 20  0]
 [ 0  0  0  3  0]]


#**Model 5: NN model- Sequential with Batchnormalisation**

In [122]:
def create_batchnor_model():
    model_nn=tf.keras.models.Sequential()

    model_nn.add(Dense(50,input_shape=(5000,)))
    model_nn.add(tf.keras.layers.BatchNormalization())
    model_nn.add(Activation('relu'))

    model_nn.add(Dense(50))
    model_nn.add(tf.keras.layers.BatchNormalization())
    model_nn.add(Activation('relu'))

    model_nn.add(Dense(50))
    model_nn.add(tf.keras.layers.BatchNormalization())
    model_nn.add(Activation('relu'))

    model_nn.add(Dense(50))
    model_nn.add(tf.keras.layers.BatchNormalization())
    model_nn.add(Activation('relu'))

    model_nn.add(Dense(1))
    model_nn.add(Activation('softmax'))

    sgd=optimizers.SGD(lr=0.001)
    model_nn.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy'])

    return model_nn

In [123]:
model_nn=create_batchnor_model()
history_nn=model_nn.fit(X_train,y_train,batch_size=100,epochs=5,verbose=0)

In [124]:
train_result_nn=model_nn.evaluate(X_train,y_train)
print('Train loss:',train_result_nn[0])
print('Train Accuracy:',train_result_nn[1])

11/11 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 0.2059
Train loss: 0.0
Train Accuracy: 0.20588235557079315


In [125]:
test_result_nn=model_nn.evaluate(X_test,y_test)
print('Test loss:',test_result_nn[0])
print('Test Accuracy:',test_result_nn[1])

3/3 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 0.2941
Test loss: 0.0
Test Accuracy: 0.29411765933036804


In [126]:

y_pred_nn=model_nn.predict(X_test)

In [127]:
NN_acc=round(accuracy_score(y_test,y_pred_nn)*100,2)
print('Test Accuracy Score : % :',NN_acc)
NN_prec=precision_score(y_test,y_pred_nn,average='micro')
print('Precision Score',NN_prec)
NN_f1=f1_score(y_test,y_pred_nn,average='micro')
print('F1 Score :',NN_f1)
NNrecall=recall_score(y_test,y_pred_nn,average='micro')
print('Recall Score:',NNrecall)

Test Accuracy Score : % : 29.41
Precision Score 0.29411764705882354
F1 Score : 0.29411764705882354
Recall Score: 0.29411764705882354


In [128]:
print(classification_report(y_test,y_pred_nn))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.29      1.00      0.45        25
           2       0.00      0.00      0.00        14
           3       0.00      0.00      0.00        33
           4       0.00      0.00      0.00         3

    accuracy                           0.29        85
   macro avg       0.06      0.20      0.09        85
weighted avg       0.09      0.29      0.13        85



In [129]:
confusn_m_NN= confusion_matrix(y_test,y_pred_nn)
print(confusn_m_NN)

[[ 0 10  0  0  0]
 [ 0 25  0  0  0]
 [ 0 14  0  0  0]
 [ 0 33  0  0  0]
 [ 0  3  0  0  0]]


In [130]:
model_nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                250050    
                                                                 
 batch_normalization (BatchN  (None, 50)               200       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 50)                2550      
                                                                 
 batch_normalization_1 (Batc  (None, 50)               200       
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 50)                0

In [131]:
models=pd.DataFrame({'Model':['Naive Bayes','Logistic Regression','Random Forest','Random Forest with MinMaxScaler','NN-Sequential'],
                   'Training Accuracy Score':[NB_train_score,LoR_train_score,RF_train_score,modelRF_SSTrain_Score,train_result_nn[1]],
                    'Testing Accuracy Score':[NB_test_score,LoR_test_score,RF_test_score,modelRF_SSTest_Score,test_result_nn[1]],
                    'Test Prediction Score':[NBacc,LRacc,RFacc,RF2_acc,NN_acc]})
models.sort_index()

,Model,Training Accuracy Score,Testing Accuracy Score,Test Prediction Score
0,Naive Bayes,0.976471,0.423529,42.35
1,Logistic Regression,0.997059,0.423529,42.35
2,Random Forest,0.997059,0.364706,36.47
3,Random Forest with MinMaxScaler,0.997059,0.388235,38.82
4,NN-Sequential,0.205882,0.294118,29.41


In [132]:
models=pd.DataFrame({'Model':['Naive Bayes','Logistic Regression','Random Forest','Random Forest with MinMaxScaler','NN-Sequential'],
                   'Confusion Matrix':[confusn_m_NB,confusn_m_LR,confusn_m_RF,confusn_m_RFwithScaling,confusn_m_NN]})
models.sort_index()

,Model,Confusion Matrix
0,Naive Bayes,"[[5, 1, 2, 2, 0], [1, 7, 9, 8, 0], [0, 3, 6, 5..."
1,Logistic Regression,"[[4, 1, 2, 3, 0], [1, 8, 9, 7, 0], [0, 6, 4, 4..."
2,Random Forest,"[[4, 1, 3, 2, 0], [0, 4, 16, 5, 0], [0, 1, 6, ..."
3,Random Forest with MinMaxScaler,"[[4, 1, 2, 3, 0], [1, 2, 14, 8, 0], [0, 3, 7, ..."
4,NN-Sequential,"[[0, 10, 0, 0, 0], [0, 25, 0, 0, 0], [0, 14, 0..."


###**• Select the final best trained model along with your detailed comments for selecting this model.**

Observations: 1.From above accuracy and confusion matrix table, we can say that Logistic Regression is the best model. Because it shows good accuracy score for traing and testing. 
2. RandomForest is equally good but it shows overfitting problem with training score.
3. We compared the confusion matrix,recall, precision, f1 score of both and made a conclusion that Logistic Regression is the best model for this problem. So, we decided to pickle the Logistic Regression model.

#**• Pickle the selected model for future use.**

In [133]:
import pickle
pkl_filename="Pickle_LR_Model.pkl"
with open(pkl_filename,'wb')as file:
    pickle.dump(LoR1,file)

###**• Import the future data file. Use the same to perform the prediction using the best chosen model from above.**

In [134]:
with open(pkl_filename,'rb') as file:
    Pickled_LR_Model=pickle.load(file)
Pickled_LR_Model

LogisticRegression(solver='liblinear')

#**Model 5: LSTM with Glove Embedding**

In [184]:
# Select input and output features
#X_text = feature_df['Description']
#y_text = feature_df['Potential Accident Level']

In [ ]:
# Encode labels in column 'Accident Level'.
y_text = LabelEncoder().fit_transform(y_text)

In [ ]:
# Divide our data into testing and training sets:
X_text_train, X_text_test, y_text_train, y_text_test = train_test_split(X_text, y_text, test_size = 0.20, random_state = 1, stratify = y_text)

In [ ]:
print('X_text_train shape : ({0})'.format(X_text_train.shape[0]))
print('y_text_train shape : ({0},)'.format(y_text_train.shape[0]))
print('X_text_test shape : ({0})'.format(X_text_test.shape[0]))
print('y_text_test shape : ({0},)'.format(y_text_test.shape[0]))

In [ ]:
# Convert both the training and test labels into one-hot encoded vectors:
from keras.utils import np_utils
y_text_train = np_utils.to_categorical(y_text_train)
y_text_test = np_utils.to_categorical(y_text_test)

In [ ]:
# The first step in word embeddings is to convert the words into thier corresponding numeric indexes.
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_text_train)

X_text_train = tokenizer.texts_to_sequences(X_text_train)
X_text_test = tokenizer.texts_to_sequences(X_text_test)

In [ ]:
# Sentences can have different lengths, and therefore the sequences returned by the Tokenizer class also consist of variable lengths.
# We need to pad the our sequences using the max length.
vocab_size = len(tokenizer.word_index) + 1
print("vocab_size:", vocab_size)

maxlen = 200
# this takes our sentences and replaces each word with an integer
# we then pad the sequences so they're all the same length (sequence_length)
X_text_train = pad_sequences(X_text_train, padding='post', maxlen=maxlen)
X_text_test = pad_sequences(X_text_test, padding='post', maxlen=maxlen)

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
!unzip glove*.zip

In [ ]:
!ls
!pwd

In [ ]:
#Using glove embedding from 200d file, which is imported locally into kaggle.
from numpy import array
from numpy import asarray
from numpy import zeros
embeddings_dictionary = dict()

glove_file = open('/content/glove.6B.200d.txt', encoding="utf8")
for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()
print('Found %s word vectors.' % len(embeddings_dictionary))

In [ ]:
#Creating embedding matrix
# first create a matrix of zeros, this is our embedding matrix

embedding_matrix = np.zeros((vocab_size, embedding_size))

# for each word in out tokenizer lets try to find that work in our glove model
# we found the word - add that words vector to the matrix

for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

len(embeddings_dictionary.values())

In [ ]:
# Reproduce the results
def reset_random_seeds():
   os.environ['PYTHONHASHSEED']=str(7)
   #np.random.seed(7)
   #random.seed(7)
   tf.random.set_seed(7)

In [ ]:
from tensorflow.keras.layers import Flatten, Activation, Dense, LSTM, BatchNormalization, Embedding, Dropout, Flatten, Bidirectional, GlobalMaxPool1D

In [ ]:
embedding_matrix.shape

In [ ]:
deep_inputs

In [ ]:
reset_random_seeds()
embedding_size = 200

deep_inputs = 200
# Build a LSTM Neural Network
deep_inputs = Input(shape=(maxlen,))

#Adding emdedding layer
#Embedding, we need to specify the embeddings matrix (the set of vectors representing our words) I'm going to make them trainable - this means they can be modified during training if something more accurate is found
embedding_layer = Embedding(vocab_size, embedding_size, weights=[embedding_matrix], trainable=False)(deep_inputs)
#add bidirectional LSTM with 128 nodes.
LSTM_Layer_1 = Bidirectional(LSTM(128, return_sequences = True))(embedding_layer)


#Adding a Max Pool (Downsamples the input representation by taking the maximum value over the time dimension)
max_pool_layer_1 = GlobalMaxPool1D()(LSTM_Layer_1)
#Adding Drop out layer ( Dropout is a technique used to prevent a model from overfitting. Dropout works by randomly setting the outgoing edges of hidden units (neurons that make up hidden layers) to 0 at each update of the training phase.)
drop_out_layer_1 = Dropout(0.5, input_shape = (256,))(max_pool_layer_1)


#Adding a dense layer of 128 nodes with activation as relu
dense_layer_1 = Dense(128, activation = 'relu')(drop_out_layer_1)
#Adding Drop out layer 
drop_out_layer_2 = Dropout(0.5, input_shape = (128,))(dense_layer_1)


#Adding a dense layer of 64 nodes with activation as relu
dense_layer_2 = Dense(64, activation = 'relu')(drop_out_layer_2)
#Adding Drop out layer 
drop_out_layer_3 = Dropout(0.5, input_shape = (64,))(dense_layer_2)

#Adding a dense layer of 32 nodes with activation as relu
dense_layer_3 = Dense(32, activation = 'relu')(drop_out_layer_3)
#Adding Drop out layer 
drop_out_layer_4 = Dropout(0.5, input_shape = (32,))(dense_layer_3)

#Adding a dense layer of 10 nodes with activation as relu
dense_layer_4 = Dense(10, activation = 'relu')(drop_out_layer_4)
#Adding Drop out layer 
drop_out_layer_5 = Dropout(0.5, input_shape = (10,))(dense_layer_4)

#Adding a output layer of 5 classes with softmax as activatin function
dense_layer_5 = Dense(5, activation='softmax')(drop_out_layer_5)

#Consutruct model with SGD and learning rate as 0.01 and categorical cross entropy for this classification problem.
model = Model(inputs=deep_inputs, outputs=dense_layer_5)

model.compile(loss='categorical_crossentropy',metrics=['acc'])

In [ ]:
class Metrics(tf.keras.callbacks.Callback):

    def __init__(self, validation_data=()):
        super().__init__()
        self.validation_data = validation_data

    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []

    def on_epoch_end(self, epoch, logs={}):
        xVal, yVal, target_type = self.validation_data
        if target_type == 'multi_class':
          val_predict_classes = model.predict_classes(xVal, verbose=0) # Multiclass
        else:
          val_predict_classes = (np.asarray(self.model.predict(xVal))).round() # Multilabel
        
        
        val_targ = yVal

        _val_f1 = f1_score(val_targ, val_predict_classes, average='micro')
        _val_recall = recall_score(val_targ, val_predict_classes, average='micro')
        _val_precision = precision_score(val_targ, val_predict_classes, average='micro')
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        #print("— train_f1: %f — train_precision: %f — train_recall %f" % (_val_f1, _val_precision, _val_recall))
        return

In [ ]:
from keras.callbacks import ReduceLROnPlateau
from sklearn import metrics
# Use earlystopping
# callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, min_delta=0.001)
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=7, min_delta=1E-3)

#TO  Reduce learning rate when a metric has stopped improving.
rlrp = ReduceLROnPlateau(monitor='val_loss', factor=0.0001, patience=5, min_delta=1E-4)

target_type = 'multi_label'
metrics = Metrics(validation_data=(X_text_train, y_text_train, target_type))

# fit the keras model on the dataset
training_history = model.fit(X_text_train, y_text_train, epochs=100, batch_size=8, verbose=2, validation_data=(X_text_test, y_text_test), callbacks=[rlrp, metrics])
                                                                                                                                                                                                                                                                                                       

In [ ]:
# evaluate the keras model
_, train_accuracy = model.evaluate(X_text_train, y_text_train, batch_size=8, verbose=0)
_, test_accuracy = model.evaluate(X_text_test, y_text_test, batch_size=8, verbose=0)

print('Train accuracy: %.2f' % (train_accuracy*100))
print('Test accuracy: %.2f' % (test_accuracy*100))

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
import pickle
pkl_filename="Pickle_LSTM_Model.pkl"
with open(pkl_filename,'wb')as file:
    pickle.dump(model,file)

#**Model 6: XLNet Model**

In [135]:
import os
os.environ['CUDA_LAUNCH_BLOCKING']='1'

In [136]:
!pip install simpletransformers

     |████████████████████████████████| 249 kB 5.3 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 1.2 MB 78.7 MB/s 
     |████████████████████████████████| 325 kB 79.5 MB/s 
     |████████████████████████████████| 3.8 MB 69.5 MB/s 
     |████████████████████████████████| 6.5 MB 68.2 MB/s 
     |████████████████████████████████| 1.7 MB 75.4 MB/s 
     |████████████████████████████████| 10.1 MB 41.6 MB/s 
     |████████████████████████████████| 895 kB 70.7 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 181 kB 82.8 MB/s 
     |████████████████████████████████| 144 kB 72.0 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 1.1 MB 73.4 MB/s 
     |████████████████████████████████| 212 kB 83.1 MB/s 
     |████████████████████████████████| 136 kB 85.1 MB/s 
     |████████████████████████████████| 138 kB 80.6 MB/s 
     |██████████████

In [137]:
!pip install h5py

In [138]:
!pip install wandb --upgrade

In [139]:
feature_df=feature_df[['Cleaned_Description','Potential Accident Level']]
feature_df['Potential Accident Level']=feature_df['Potential Accident Level'].astype('category').cat.codes
#For SimpleTransformers, the 'text' column should be the first column
feature_df.columns=['text','label'] #We need this for simpletransformers


In [140]:
feature_df.head(2)

,text,label
0,removing drill rod jumbo maintenance superviso...,3
1,activation sodium sulphide pump piping uncoupl...,3


In [141]:
# number of unique categories in the target column
num_labels=feature_df['label'].nunique()
num_labels,sorted(feature_df['label'].unique())

(6, [0, 1, 2, 3, 4, 5])

In [142]:
feature_df['label'].value_counts()

3    143
2    106
1     95
0     49
4     31
5      1
Name: label, dtype: int64

This is one of the biggest challenges in this dataset as it is highly imbalanced with majority of data falling under the ‘Potential Accident Level’ feature with label 3. Most machine learning techniques will ignore the minority classes, and performs very poorly in predicting them even though these minority classes are also important. The data is not sufficient for a model to be trained. To overcome this, we will perform data augmentation in the next step.

##**Data Augmentation**

Data augmentation is done by replacing synonyms of words for the records belonging to all the minority classes. For finding synonyms 'wordnet' module is used from the 'nltk.corpus' library.

In [143]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [144]:
from nltk.corpus import wordnet

#Define a function to find synonyms of words in a text.
def get_synonyms(word):

    synonyms=set()

    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            synonym=l.name().replace("_", " ").replace("-", " ").lower()
            synonym="".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
            synonyms.add(synonym)
    if word in synonyms:
        synonyms.remove(word)

    return list(synonyms)

In [145]:
from nltk.corpus import stopwords

stop_words=[]
for w in stopwords.words('english'):
    stop_words.append(w)
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Once the synonym is generated, we'll define a function to replace the words in 
a sentence with their synonyms for a specified number of eamples. Following is a snippet of the function that replaces words with synonyms:


In [146]:
import random

def synonym_replacement(words,n):
  
    words=words.split()

    new_words=words.copy()
    random_word_list=list(set([word for word in words if word not in stop_words]))
    random.shuffle(random_word_list)
    num_replaced=0

    for random_word in random_word_list:
        synonyms=get_synonyms(random_word)

        if len(synonyms)>=1:
            synonym=random.choice(list(synonyms))
            new_words=[synonym if word==random_word else word for word in new_words]
            num_replaced+=1

        if num_replaced>=n: #only replace upto n words
           break

    sentence=' '.join(new_words)

    return sentence

Let's now test how the augmented data looks like. For this, we'll fetch a random sample from the dataframe.

In [147]:
# Displaying a sample text to display the synthetic data
trial_sent=feature_df['text'][6]
print(trial_sent)

collaborator reports street holding left hand volumetric balloon slipped placing hand ground volumetric balloon ended breaking caused small wound left hand


Now, let's view 3 different augmented data of the sample we've chosen.

In [148]:
# Creates 3 augmented sentences per data 
for n in range(3):
    print(f"Example of synonym replacement: {synonym_replacement(trial_sent,n)}")

Example of synonym replacement: collaborator reports street hold in left hand volumetric balloon slipped placing hand ground volumetric balloon ended breaking caused small wound left hand
Example of synonym replacement: collaborator reports street holding left hand volumetric balloon slipped placing hand ground volumetric balloon ended crack caused small wound left hand
Example of synonym replacement: collaborator reports street holding left hand volumetric balloon slipped site hand ground volumetric balloon concluded breaking caused small wound left hand


We can see that few words such as 'ended' was replaced by its synonym 'terminated'. 

Now, we'll create a copy of the original dataframe and create augmented data of all the records of minority classes (II, III, IV and V). We'll define a function that generates augmented sentences depending on the number of records each minority class contains. Since the class V has the least umber of records, we'll create more augmented records for it and also ensure that these classes are not oversampled.

In [149]:
#df_aug = df.copy() # creating a copy of the dataframe
df_aug=feature_df.copy()

In [150]:
augmented_sentences=[]
augmented_sentences_labels=[]

# Generate synthesized data for minority classes (II, III, IV and V) using the synonym_replacement(words, n) function.
for i in df_aug.index:
  if df_aug['label'][i]==1:  # Accident Level II
    for n in range(6):
      temps=synonym_replacement(df_aug['text'][i],n)
      augmented_sentences.append(temps)
      augmented_sentences_labels.append(df_aug['label'][i])
  elif df_aug['label'][i]==2:  # Accident Level III
    for n in range(9):
      temps=synonym_replacement(df_aug['text'][i],n)
      augmented_sentences.append(temps)
      augmented_sentences_labels.append(df_aug['label'][i])
  elif df_aug['label'][i]==3:  # Accident Level IV
    for n in range(9):
      temps=synonym_replacement(df_aug['text'][i],n)
      augmented_sentences.append(temps)
      augmented_sentences_labels.append(df_aug['label'][i])
  elif df_aug['label'][i]==4:  # Accident Level V
    for n in range(37):
      temps=synonym_replacement(df_aug['text'][i],n)
      augmented_sentences.append(temps)
      augmented_sentences_labels.append(df_aug['label'][i])

In [151]:
# creating dataframes for augmented text and their labels
augmented_sentences=pd.DataFrame(augmented_sentences)
augmented_sentences_labels=pd.DataFrame(augmented_sentences_labels)

In [152]:
aug_df=pd.concat([augmented_sentences,augmented_sentences_labels],axis=1,ignore_index=True)
aug_df.rename(columns={0:'text',1:'label'},inplace=True)
display(aug_df.head(2).append(aug_df.tail(2)))

,text,label
0,removing drill rod jumbo maintenance superviso...,3
1,removing drill rod jumbo maintenance superviso...,3
3956,assistant cleaned floor module es central cliq...,1
3957,assistant cleaned blow out of the water module...,1


In [153]:
# Appending the augmented dataframe to the original dataframe
df_aug=df_aug.append([aug_df],ignore_index=True)

In [154]:
# Display the shapes of original, augmented and combined dataframes
feature_df.shape, aug_df.shape,df_aug.shape

((425, 2), (3958, 2), (4383, 2))

In [155]:
df_aug.columns

Index(['text', 'label'], dtype='object')

In [156]:
# shift column 'text' to first position
first_column=df_aug.pop('text')
  
# insert column using insert(position,column_name, first_column) function
df_aug.insert(0,'text',first_column)  
df_aug.head(1)

,text,label
0,removing drill rod jumbo maintenance superviso...,3


Splitting into Train-Test sets


In [157]:
from sklearn.model_selection import train_test_split

df_train,df_test=train_test_split(df_aug,test_size=0.20,random_state=1)

print(f"df             : {df_aug.shape}")
print(f"\ndf_train : {df_train.shape}")
print(f"\ndf_test       : {df_test.shape}")

df_train.head(2)

df             : (4383, 2)

df_train : (3506, 2)

df_test       : (877, 2)


,text,label
2401,check list country ​​the survey operator slue ...,1
2934,performing crusade debar make room place calib...,3


Let's quickly check if GPU is available in Colab.

In [158]:
# To check if GPU is running or not, run the following command
!nvidia-smi  

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [159]:
import torch
torch.cuda.is_available()

False

Data Modelling using SimpleTransformers library

In [160]:
!rm -rf outputs

In [161]:
!pip install setuptools==59.5.0

     |████████████████████████████████| 952 kB 5.1 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 61.3.1
    Uninstalling setuptools-61.3.1:
      Successfully uninstalled setuptools-61.3.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
ipykernel 6.11.0 requires setuptools>=60, but you have setuptools 59.5.0 which is incompatible.
google-colab 1.0.0 requires ipykernel~=4.10, but you have ipykernel 6.11.0 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.32.0 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
google-colab 1.0.0 requires tornado~=5.1.0; python_version >= "3.0", but you have tornado 6.1 which is incompatible.


In [162]:
from distutils.version import LooseVersion
#from setuptools import distutils; distuils.version.LooseVersion

In [163]:
from simpletransformers.classification import ClassificationModel,ClassificationArgs
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger=logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

1. "xlnet" : "xlnet-base-cased"

Defining ModelArgs for XLNet Model

In [164]:
#The modelArgs(train_args) dataclass contains all the global options set to their default values

model_type='xlnet'
model_name='xlnet-base-cased'

train_args={
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,

    #Paths
    #"output_dir": f"/content/drive/MyDrive/Colab Notebooks/Capstone Project/outputs/xlnet/final",
    #"best_model_dir": f"/content/drive/MyDrive/Colab Notebooks/Capstone Project/outputs/xlnet/best_model",
    "output_dir": f"/content/drive/MyDrive/AIML/Capstone Project Data/final",
    "best_model_dir": f"/content/drive/MyDrive/AIML/Capstone Project Data/best_model",

    #size
    "train_batch_size": 128,
    "max_seq_length": 128,
    "num_train_epochs": 2,

    #evaluation
    "evaluation_during_training": False,
    "evaluation_during_tarining_steps": 1000,
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "evaluate_during_training_verbose": False,
    "sliding_window": False,
    "eval_batch_size": 64,
    "gradient_accumulation_steps": 1,
   
}

if model_type=="xlnet":
   train_args['train_batch_size']=64
   train_args['gradient_accumulation_steps']=2

Initializing a classification model for XLNet model

In [165]:
model=ClassificationModel(model_type,model_name,args=train_args,num_labels=6,use_cuda=False,)

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.weight', 'sequence_summary.summary.bias', 'logits_proj.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Training the XLNet classification model

In [166]:
!pip install torch==1.10.0 torchvision==0.7.0

     |████████████████████████████████| 5.9 MB 4.8 MB/s 
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     |██████████████████████████████▎ | 834.1 MB 1.1 MB/s eta 0:00:43tcmalloc: large alloc 1147494400 bytes == 0x5597a7f48000 @  0x7fe8bd11b615 0x55976de9217c 0x55976df7247a 0x55976de94f9d 0x55976df86d4d 0x55976df08ec8 0x55976df03a2e 0x55976de9688a 0x55976df08d30 0x55976df03a2e 0x55976de9688a 0x55976df05719 0x55976df87b76 0x55976df04d95 0x55976df87b76 0x55976df04d95 0x55976df87b76 0x55976df04d95 0x55976de96ce9 0x55976deda579 0x55976de95902 0x55976df08c4d 0x55976df03a2e 0x55976de9688a 0x55976df05719 0x55976df03a2e 0x55976de9688a 0x55976df048f6 0x55976de967aa 0x55976df04b4f 0x55976df03a2e
     |████████████████████████████████| 881.9 MB 16 kB/s 
INFO: pip is looking at multiple versions of <Python from Requires-Python> to determine which version is compatible with other requirements. This cou

In [167]:
import subprocess,sys
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'my_package'])

0

In [168]:
model.train_model(df_train,eval_df=None)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/3506 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_xlnet_128_6_2


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/55 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/55 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of xlnet model complete. Saved to /content/drive/MyDrive/AIML/Capstone Project Data/final.


(54, 1.1761057376861572)

Loading the saved model

The model is saved in the 'output_dir' defined in ModelArgs

In [171]:
load_model_xlnet=ClassificationModel(model_type,'/content/drive/MyDrive/AIML/Capstone Project Data/final',use_cuda=False,)

Evaluating XLNet model

In [176]:
#from sklearn.metrics import precision_recall_fscore_support

#def compute_metrcis(y_true,y_pred):
 # precision,recall,f1, _=precision_recall_fscore_support(y_true,y_pred,average='micro')
 # return{
  #    'f1':f1,
   #   'precision':precision,
    #  'recall':recall
  #}

In [174]:
import sklearn
result_xlnet,model_outputs,wrong_predictions=load_model_xlnet.eval_model(df_test,acc=sklearn.metrics.accuracy_score,)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/877 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_xlnet_128_6_2


Running Evaluation:   0%|          | 0/14 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.49886707788908424, 'acc': 0.6259977194982896, 'eval_loss': 0.9233080574444362}


In [175]:
#Displaying the model metrics
result_xlnet

{'acc': 0.6259977194982896,
 'eval_loss': 0.9233080574444362,
 'mcc': 0.49886707788908424}